In [2]:
import string
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:

with open('datasets/allan_poe.txt','r',encoding='utf-8') as file:
    allan_poe = file.read()



with open('datasets/robert_frost.txt','r',encoding='utf-8') as file:
    robert_frost= file.read()

input_files = [allan_poe, robert_frost]

In [4]:
input_texts = []
labels = []



for label,collection in enumerate(input_files):
    #print(f"{collection} corresponds to label {label}")
    
   
    #Loop each line
    for index,line in enumerate(collection.split("\n")):
        
        #lowercase 
        line = line.lower()
        if line:
            # remove puncuation
            line = line.translate(str.maketrans('','', string.punctuation))
        
        
            input_texts.append(line)
            labels.append(label)
    
    
        

In [5]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)
print(len(Ytrain))
print(len(Ytest))

1618
540


In [6]:
print(train_text[:3])
print(Ytrain[:5])

['let people in than i can keep them out ', 'its harmless mother hears it in the night', 'on last looking into his hesperides']
[1, 1, 1, 1, 1]


In [7]:
print(bool(Ytrain[0]))

True


In [8]:
idx = 1
word2idx = {'<unk>':0}

In [9]:
# Populate word2idx
# each line
for line in train_text:
    # tokenize with string split
    for token in line.split():
        if token not in  word2idx:
            word2idx[token] = idx
            idx += 1

In [10]:
len(word2idx)

2508

In [11]:
#convert words to idx
token_train_text = []
token_test_text = []


for line in train_text:
    tokens = line.split()
    line_as_int = [word2idx[word] for word in tokens]
    token_train_text.append(line_as_int)
    
for line in test_text:
    tokens = line.split()
    line_as_int = []
    for word in tokens:
        if word not in word2idx:
            line_as_int.append(word2idx['<unk>'])
        else:
            line_as_int.append(word2idx[word])
                 

    token_test_text.append(line_as_int)

In [12]:
token_train_text[100:105]

[[143, 39, 386, 308, 39, 117, 74, 387, 41, 211],
 [56, 388, 64, 65, 389, 60],
 [390, 39, 118, 132, 117, 391, 245, 266, 35, 195],
 [143, 392, 15, 144, 32, 393, 9],
 [75, 75, 394, 395, 396, 395, 397, 4, 398]]

In [13]:
### Build Markov models ###

# Step 1: set Ai and pi
V = len(word2idx)

A0 = np.ones((V,V))
pi0 = np.ones(V)


A1 = np.ones((V,V))
pi1 = np.ones(V)

# Step 2: Compute A, pi0


def compute_counts(text_as_int,A,pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            if last_idx is None:
                # it's the first word in a sentence
                pi[idx] += 1
                
            else:
                # the last word exists, so count a transtision
                A[last_idx.idx] += 1
                
        # update last idx
        last_idx = idx
        
        
compute_counts([t for t,y in zip(token_train_text,Ytrain) if y == 0], A0, pi0)
compute_counts([t for t,y in zip(token_train_text,Ytrain) if y == 0], A1, pi1)


In [ ]:
# Normalize counts for A and i
A0 /= A0.sum(axis=1,keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1,keepdims=True)
pi1 /= pi1.sum()